In [ ]:
%%capture
!pip install "dask[distributed]"
!pip install planetary-computer

## Imports

In [1]:
import geopandas as gpd
import shapely.geometry
from shapely.geometry import box, shape
import stackstac
import dask
import xarray as xr
import os
import numpy as np
import leafmap
import pystac_client
import shapely
from rioxarray.exceptions import NoDataInBounds
import rioxarray as rxr

## Making grids, skip if you already have the grids

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
zambia_boundary = world[world["name"] == "Zambia"]

utm_crs = zambia_boundary.estimate_utm_crs()
zambia_boundary = zambia_boundary.to_crs(utm_crs)

# 10 meters per pixel → 2240m grid size
grid_size = 2240

minx, miny, maxx, maxy = zambia_boundary.total_bounds

grid_tiles = []
for x in np.arange(minx, maxx, grid_size):
    for y in np.arange(miny, maxy, grid_size):
        grid_tiles.append(box(x, y, x + grid_size, y + grid_size))

grid_gdf = gpd.GeoDataFrame({"geometry": grid_tiles}, crs=utm_crs)

zambia_grid = gpd.clip(grid_gdf, zambia_boundary)

zambia_grid = zambia_grid.to_crs(epsg=4326)

zambia_grid.to_file("zambia_grid.geojson", driver="GeoJSON")

print(f"Created {len(zambia_grid)} grid tiles.")

In [2]:
zambia_grid = gpd.read_file("zambia_grid.geojson")

zambia_random_grids = zambia_grid.sample(n=20, random_state=42)
zambia_random_grids.to_file("zambia_random_grids.geojson", driver="GeoJSON")

### check the grids

In [ ]:
zambia_random_grids = gpd.read_file("zambia_random_grids.geojson")
zambia_grid = zambia_random_grids#.to_crs("EPSG:4326")

ma = leafmap.Map(center=[-13.1339, 27.8493], zoom=6)
ma.add_basemap('SATELLITE')
ma.add_geojson(zambia_random_grids)
ma

## Downloading S2 pipeline

### Load and Prepare Zambia Grids

In [ ]:
zambia_boundary = gpd.read_file("zambia_boundary.geojson")
zambia_boundary = zambia_boundary.to_crs("EPSG:4326")
bbox = tuple(zambia_boundary.total_bounds)

print("Bounding Box:", bbox)

### Search, Stack, Composite, and Save

### Test

In [ ]:
catalog = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime="2020-01-01/2020-01-30",
    query={"eo:cloud_cover": {"lt": 60}},
    limit=10,
)

In [ ]:
items = search.get_all_items()
len(items)

In [ ]:
items[0]

In [ ]:
items[0].properties['proj:code']

In [ ]:
zambia_random_grids = gpd.read_file("zambia_random_grids.geojson")
zambia_grid = zambia_random_grids.to_crs("EPSG:4326")

In [ ]:
m = leafmap.Map(center=[-13.1339, 27.8493], zoom=5, height="600px")
m.add_geojson(gpd.GeoDataFrame.from_features(items.to_dict(), crs="EPSG:4326"))
#m.add_geojson(items.to_dict())
m.add_geojson(zambia_grid, style={"color": "red", "fillOpacity": 0.3})
m

In [ ]:
import glob

m = leafmap.Map(center=[-13.1339, 27.8493], zoom=5, height="600px")
tif_files = glob.glob(os.path.join(output_folder, "*.tif"))

for tif_file in tif_files:
    m.add_raster(tif_file, colormap="viridis", opacity=0.7)
    m.add_geojson(zambia_grid, style={"color": "red", "fillOpacity": 0.3})

m

## Downloading pipeline

In [ ]:
# Define target months
target_months = ["11-2020", "01-2021", "03-2021", "05-2021", 
                 "11-2021", "01-2022", "03-2022", "05-2022"]

# Open STAC Catalog
catalog = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")

# Output Directory
output_folder = "Sentinel2_UTM_TimeSeries"
os.makedirs(output_folder, exist_ok=True)

# Sentinel-2 Bands (including SWIR1 and RedEdge1) + Scene Classification Layer (SCL)
assets = ['blue', 'green', 'nir', 'red', 'swir1', 'rededge1', 'scl']

In [ ]:
# Function to determine UTM zone from longitude
def get_utm_zone(longitude):
    return int((longitude + 180) / 6) + 1

In [ ]:
# Process each grid cell
for index, row in zambia_grid.iterrows():
    grid_geom = row.geometry
    centroid_lon = grid_geom.centroid.x
    utm_zone = get_utm_zone(centroid_lon)
    utm_epsg = 32700 + utm_zone  # UTM Southern Hemisphere

    print(f"Processing Grid {index} - Expected UTM Zone: EPSG:{utm_epsg}")

    stack_list = []
    reference_array = None  # Store reference dataset for alignment (S2 and S1)

    for month in target_months:
        month_number, year = month.split("-")  
        date_range = f"{year}-{month_number}-01/{year}-{month_number}-30"

        # Search Sentinel-2 Images
        search = catalog.search(
            collections=["sentinel-2-l2a"],
            intersects=grid_geom,
            datetime=date_range,
            limit=10
        )

        items = search.get_all_items()
        if not items:
            print(f"Skipping {month} for Grid {index}: No matching images.")
            continue

        # Extract CRS from first item using `proj:code`
        first_item = items[0]
        if "proj:code" in first_item.properties:
            sentinel_epsg = int(first_item.properties["proj:code"].replace("EPSG:", ""))
        else:
            print(f"Skipping {month} for Grid {index}: No CRS found.")
            continue

        print(f"Using CRS EPSG:{sentinel_epsg} for {month}")

        # Stack Images **Explicitly setting the CRS**
        stack = stackstac.stack(items, assets=assets, epsg=sentinel_epsg, resolution=10)

        # Reproject the grid geometry to match Sentinel-2 CRS
        grid_utm = gpd.GeoSeries(grid_geom, crs=zambia_grid.crs).to_crs(f"EPSG:{sentinel_epsg}").geometry[0]

        # Clip using the reprojected grid
        stack_clipped = stack.rio.clip([grid_utm], f"EPSG:{sentinel_epsg}", drop=True)

        # Mask Invalid Pixels Based on SCL
        scl_mask = (stack_clipped.sel(band="scl") != 0) & \
                   (stack_clipped.sel(band="scl") != 1) & \
                   (stack_clipped.sel(band="scl") != 3) & \
                   (stack_clipped.sel(band="scl") < 8)

        stack_filtered = stack_clipped.where(scl_mask, np.nan)

        # Compute Median Composite for the Month
        stack_median = stack_filtered.median(dim="time", keep_attrs=True)

        # Store reference shape from the first image
        if reference_array is None:
            reference_array = stack_median

        # Ensure all images have the same shape before stacking
        if stack_median.shape != reference_array.shape:
            stack_median = stack_median.rio.reproject_match(reference_array)

        # Align coordinates before appending to stack_list
        stack_median = stack_median.assign_coords(reference_array.coords)

        # Ensure final chip is exactly 224 x 224 pixels
        stack_median = stack_median.rio.reproject(
            dst_crs=f"EPSG:{sentinel_epsg}",
            shape=(224, 224),
            resampling=1  # Nearest neighbor resampling
        )

        stack_list.append(stack_median)

    if stack_list:
        # Convert list to xarray DataArray before writing
        time_series_stack = xr.concat(stack_list, dim="time", coords="minimal", compat="override")
        time_series_stack.attrs["crs"] = f"EPSG:{sentinel_epsg}"

        # Remove SCL Band before reshaping
        if "scl" in time_series_stack.coords["band"].values:
            time_series_stack = time_series_stack.sel(band=time_series_stack.band != "scl")

        # Reshape from 4D (time, band, height, width) -> 3D (new_band, height, width)
        time_series_stack = time_series_stack.stack(new_band=("time", "band"))

        # Remove time dimension before transposing
        if "time" in time_series_stack.dims:
            time_series_stack = time_series_stack.isel(time=0)

        # Transpose to (band, height, width) format
        time_series_stack = time_series_stack.transpose("new_band", "y", "x")

        # Extract Sentinel-2 item ID from the first image in the list
        sentinel_item_id = first_item.id  

        # Construct the output filename with Sentinel-2 item ID
        output_filename = f"{output_folder}/{sentinel_item_id}_Grid_{index}_UTM_{utm_zone}.tif"

        # Save the raster
        time_series_stack.rio.to_raster(output_filename)

        # Print summary of final raster dimensions
        # Get the actual height and width from the xarray dataset
        h, w = time_series_stack.sizes["y"], time_series_stack.sizes["x"]
        print(f"\nFinal Image Dimensions: {time_series_stack.shape}")
        print(f"Saved time-series stack for Grid {index} in EPSG:{sentinel_epsg} as {output_filename}\n")

print("Processing complete!")
